# NLP con Deep Learning: Transformers

# - DistilBert

In [1]:
import torch
import numpy as np
import pandas as pd

In [2]:
device = "cuda:0" if torch.cuda.is_available() else "cpu" #para gpu
print(device)

cuda:0


In [3]:
df = pd.read_csv('dataset_final_actualizacion_mayus2.csv')
df

Periódico                                       Hipervínculo  \
0       ElDiario.es  https://www.eldiario.es/politica/pp-recupera-l...   
1       ElDiario.es  https://www.eldiario.es/politica/58-ciudadanos...   
2       ElDiario.es  https://www.eldiario.es/politica/diputado-ayus...   
3       ElDiario.es  https://www.eldiario.es/andalucia/almeria/psoe...   
4       ElDiario.es  https://www.eldiario.es/castilla-la-mancha/pol...   
...             ...                                                ...   
4974  HayNoticia.es  https://haynoticia.es/una-empresa-catalana-tri...   
4975  HayNoticia.es  https://haynoticia.es/hospitalizado-tras-inten...   
4976  HayNoticia.es  https://haynoticia.es/lleva-300-bolsas-de-plas...   
4977  HayNoticia.es  https://haynoticia.es/la-dgt-multara-a-los-con...   
4978  HayNoticia.es  https://haynoticia.es/una-sentencia-prohibe-a-...   

     Fecha publicación                                            Titular  \
0           2023-07-11  El PP se recupera ligeramente pero sigue sin o...   
1           2023-07-11  Un 58% de los ciudadanos critican los pactos c...   
2           2023-07-11  Un diputado de Ayuso reúne a un grupo de vícti...   
3           2023-07-11  El PSOE de Mójacar recurre ante el Constitucio...   
4           2023-07-11  La exalcaldesa de Toledo pide a Feijóo que fir...   
...                ...                                                ...   
4974        2018-11-06  Una empresa catalana triunfa con su papel higi...   
4975        2018-10-31  Hospitalizado tras intentar sacarse una muela ...   
4976        2018-10-30  Lleva 300 bolsas de plástico a Mercadona para ...   
4977        2018-10-28  La DGT multará a los conductores que no hayan ...   
4978        2018-10-25  Una Sentencia prohibe a los comercios poner vi...   

                                              Subtítulo  \
0     La estimación de escaños no se mueve en la enc...   
1     El 17,8% de quienes apuestan por Feijóo ven “m...   
2     Daniel Portero trata con urgencia de contrarre...   
3     Ya con nuevo alcalde del PP, los socialistas r...   
4     La socialista, también número dos al Congreso,...   
...                                                 ...   
4974                                                NaN   
4975                                                NaN   
4976                                                NaN   
4977                                                NaN   
4978                                                NaN   

                                                 Cuerpo           Categoría  \
0     El partido popular se recupera muy ligeramente...            politica   
1     El PP ha necesitado a Vox para recuperar buena...            politica   
2     El diputado del PP en la Asamblea de Madrid Da...            politica   
3     Mojácar retomó el ritmo político con la procla...           andalucia   
4     La exalcaldesa de Toledo y 'número dos' de la ...  castilla-la-mancha   
...                                                 ...                 ...   
4974  La empresa catalana Banys Nous de Barcelona es...        curiosidades   
4975  Sucedió el pasado lunes poco después de las 20...        curiosidades   
4976  Un joven ha sido noticia ayer tras presentarse...        curiosidades   
4977  Este domingo nos ha tocado hacer por segunda v...        curiosidades   
4978  Así se ha pronunciado la Audiencia Provincial ...        curiosidades   

      Target  
0          0  
1          0  
2          0  
3          0  
4          0  
...      ...  
4974       1  
4975       1  
4976       1  
4977       1  
4978       1  

[4979 rows x 8 columns]

In [4]:
df.Target.value_counts()

0    2506
1    2473
Name: Target, dtype: int64

In [5]:
df["Texto"] = df.Titular + " " + df.Cuerpo
df.head()

Periódico                                       Hipervínculo  \
0  ElDiario.es  https://www.eldiario.es/politica/pp-recupera-l...   
1  ElDiario.es  https://www.eldiario.es/politica/58-ciudadanos...   
2  ElDiario.es  https://www.eldiario.es/politica/diputado-ayus...   
3  ElDiario.es  https://www.eldiario.es/andalucia/almeria/psoe...   
4  ElDiario.es  https://www.eldiario.es/castilla-la-mancha/pol...   

  Fecha publicación                                            Titular  \
0        2023-07-11  El PP se recupera ligeramente pero sigue sin o...   
1        2023-07-11  Un 58% de los ciudadanos critican los pactos c...   
2        2023-07-11  Un diputado de Ayuso reúne a un grupo de vícti...   
3        2023-07-11  El PSOE de Mójacar recurre ante el Constitucio...   
4        2023-07-11  La exalcaldesa de Toledo pide a Feijóo que fir...   

                                           Subtítulo  \
0  La estimación de escaños no se mueve en la enc...   
1  El 17,8% de quienes apuestan por Feijóo ven “m...   
2  Daniel Portero trata con urgencia de contrarre...   
3  Ya con nuevo alcalde del PP, los socialistas r...   
4  La socialista, también número dos al Congreso,...   

                                              Cuerpo           Categoría  \
0  El partido popular se recupera muy ligeramente...            politica   
1  El PP ha necesitado a Vox para recuperar buena...            politica   
2  El diputado del PP en la Asamblea de Madrid Da...            politica   
3  Mojácar retomó el ritmo político con la procla...           andalucia   
4  La exalcaldesa de Toledo y 'número dos' de la ...  castilla-la-mancha   

   Target                                              Texto  
0       0  El PP se recupera ligeramente pero sigue sin o...  
1       0  Un 58% de los ciudadanos critican los pactos c...  
2       0  Un diputado de Ayuso reúne a un grupo de vícti...  
3       0  El PSOE de Mójacar recurre ante el Constitucio...  
4       0  La exalcaldesa de Toledo pide a Feijóo que fir...

In [6]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df["Texto"].tolist(), 
                                                  df["Target"].tolist(), 
                                                  test_size=0.45, 
                                                  stratify=df["Target"],
                                                  random_state=42)

# HF Transformers

In [7]:
# !pip install transformers[torch] --quiet
# !pip install accelerate -U --quiet

In [9]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("Geotrend/distilbert-base-es-cased")
model = AutoModelForSequenceClassification.from_pretrained("Geotrend/distilbert-base-es-cased", num_labels=2).to(device)

Some weights of the model checkpoint at Geotrend/distilbert-base-es-cased were not used when initializing DistilBertForSequenceClassification: ['vocab_transform.weight', 'vocab_transform.bias', 'vocab_layer_norm.bias', 'vocab_projector.bias', 'vocab_layer_norm.weight']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at Geotrend/distilbert-base-es-cased and are newly initialized: ['pre_classifier.bias', 'classifier.bias', 'pre_classifier.wei

In [39]:
torch.__version__

'2.0.1+cu117'

In [10]:
train_encodings = tokenizer(X_train, truncation=True, padding=True)
test_encodings = tokenizer(X_test, truncation=True, padding=True)

In [41]:
type(train_encodings)

transformers.tokenization_utils_base.BatchEncoding

In [42]:
train_encodings.keys()

dict_keys(['input_ids', 'attention_mask'])

In [11]:
class MyDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        #item es un diccionario que contiene tres keys: 'input_ids', 'attention_mask' y 'labels'
        #cada key contiene el tensor correspodiente al indice idx
        #item = {key: torch.tensor(val[idx]).to(device) for key, val in self.encodings.items()}
        item = {}
        item['input_ids'] = torch.tensor(self.encodings['input_ids'][idx]).to(device)
        item['attention_mask'] = torch.tensor(self.encodings['attention_mask'][idx]).to(device)
        item['labels'] = torch.tensor(self.labels[idx]).to(device)
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = MyDataset(train_encodings, y_train)
test_dataset = MyDataset(test_encodings, y_test)

In [12]:
print(len(train_dataset),len(test_dataset))

2738 2241


In [45]:
train_dataset[0]

{'input_ids': tensor([   11,   258, 24389, 10869, 22484,   205,  3935,    39,  3123,  3376,
            25,   551,   930,   638, 24718,   101,   259, 25928,  6853,   205,
          1775,  5918, 25422,   233,   222,  5253, 12951,   696,   238, 16451,
           233,   841,  3826,   315, 10722,   281, 10778,   391,   205,   210,
          1554,    25,   225,  4575,   101,   225,  6401,   205,  5509,   213,
           205,  2204,    27, 25422,   233,   222,   238,  3249,  1794,   210,
         21153,   205,  3935,  3088,   101,   205,   370, 14227,   227,  1778,
           315, 13268,  7119, 17289,   211,   210,  1503,   227,  6240,  1472,
          3521,    25,   225, 20478,   417,   233,  1305,  1417,   756,  9372,
           386,   263,  3680,  4243,  7423,  9489,   296,  1109,  2668,   205,
          5072,    30,    27, 25422,   233,   222,  2179, 18559,  5053,   205,
           370,   275,  2290, 15110, 15149,  6181,   205,  1528, 18904,  5667,
         25781,    25,  1625, 25939,   

In [46]:
!pip install evaluate --quiet

In [13]:
import evaluate
metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [25]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    num_train_epochs=3,              # total number of training epochs
    per_device_train_batch_size=6,  # batch size per device during training
    per_device_eval_batch_size=8,   # batch size for evaluation
    dataloader_pin_memory=False,     # remove if possible for faster training
    evaluation_strategy = "epoch",
    output_dir="./results"
)

trainer = Trainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=test_dataset,            # evaluation dataset
    compute_metrics=compute_metrics,
)

In [26]:
trainer.train()

TrainOutput(global_step=1371, training_loss=0.10135258383163129, metrics={'train_runtime': 1677.6753, 'train_samples_per_second': 4.896, 'train_steps_per_second': 0.817, 'total_flos': 1088087212560384.0, 'train_loss': 0.10135258383163129, 'epoch': 3.0})

In [22]:
 import torch
 torch.cuda.empty_cache()

In [17]:
test_preds = []
for i in range(len(X_test)):
    val_encoding = tokenizer(X_test[i], truncation=True, padding=True, return_tensors="pt").to(device)
    outputs = model(**val_encoding)
    logits = outputs.logits.cpu().detach().numpy()
    test_preds.append(np.argmax(logits))

In [ ]:
# Datos:
# Epochs: 1
# per_device_train_batch_size=6
# per_device_eval_batch_size=8

In [18]:
print('accuracy del train:')
trainer.evaluate(train_dataset)['eval_accuracy']

accuracy del train:


0.9737034331628927

In [19]:
print('accuracy del test:')
trainer.evaluate(test_dataset)['eval_accuracy']

accuracy del test:


0.9647478804105311

In [27]:
#---------------------------------------------

In [ ]:
# Datos:
# Epochs: 3
# per_device_train_batch_size=6
# per_device_eval_batch_size=8

In [28]:
print('accuracy del train:')
trainer.evaluate(train_dataset)['eval_accuracy']

accuracy del train:


0.9974433893352812

In [19]:
print('accuracy del test:')
trainer.evaluate(test_dataset)['eval_accuracy']

accuracy del test:


0.9647478804105311